In [151]:
# importing necessary libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [152]:
#get the url 
url = 'https://en.wikipedia.org/wiki/Demographics_of_India'
response = requests.get(url)
response

<Response [200]>

In [153]:
#create soup
soup = BeautifulSoup(response.content,'lxml')

Get table for population distribution by states / union territories

In [154]:
#required table is under tag table and class = wikitable sortable
pop_dist = soup.find_all('table', class_ = 'wikitable sortable')[4]

In [155]:
states = []
# get states names from the table    
for tag in pop_dist.find_all('td'):
    if tag.find('a'):
        states.append(tag.find('a').text)
        continue

In [156]:
print(states) 

['Uttar Pradesh', 'Maharashtra', 'Bihar', 'West Bengal', 'Madhya Pradesh', 'Tamil Nadu', 'Rajasthan', 'Karnataka', 'Gujarat', 'Andhra Pradesh', 'Odisha', 'Telangana', 'Kerala', 'Jharkhand', 'Assam', 'Punjab', 'Chhattisgarh', 'Haryana', 'Delhi', 'Jammu and Kashmir', 'Uttarakhand', 'Himachal Pradesh', 'Tripura', 'Meghalaya', 'Manipur', 'Nagaland', 'Goa', 'Arunachal Pradesh', 'Puducherry', 'Mizoram', 'Chandigarh', 'Sikkim', 'Andaman and Nicobar Islands', 'Dadra and Nagar Haveli', 'Daman and Diu', 'Lakshadweep']


In [157]:
# collect data for rest of the table 
# Respective column data is available at every 12th step.
all_data = []
for i in range(2,12):
    all_data.append(pop_dist.find_all('td')[i::12])

In [158]:
#create function to get individual column 
def get_data(all_data):
    df_list = []
    for data in all_data:
        df_list.append(int(data.text.strip().replace(',','')))
    return df_list

In [159]:
df_dict = {0 : [], 2 : [], 3 : [], 4 : [], 5 : [], 6 : [], 7 : [], 8 : [], 9 : []}

In [160]:
#call get data function and store the column in respective value of df_dict
for i in range(len(df_dict)+1):
    if i == 1:
        continue
    else:
        data = get_data(all_data[i])
        df_dict[i] = data

In [161]:
percent = []
for tag in all_data[1]:
     percent.append(float(tag.text.strip()))

In [162]:
states = pd.DataFrame(states,columns = ['states'])
percent = pd.DataFrame(percent,columns = ['percent'])
df = pd.DataFrame(df_dict)
df = pd.concat([states,df,percent],axis = 1)
df.head()

,states,0,2,3,4,5,6,7,8,9,percent
0,Uttar Pradesh,199812341,104480510,95331831,9148679,930,155111022,44470455,240928,828,16.50
1,Maharashtra,112374333,58243056,54131277,4111779,929,61545441,50827531,307713,365,9.28
2,Bihar,104099452,54278157,49821295,4456862,918,92075028,11729609,94163,1102,8.60
3,West Bengal,91276115,46809027,44467088,2341939,950,62213676,29134060,88752,1030,7.54
4,Madhya Pradesh,72626809,37612306,35014503,2597803,931,52537899,20059666,308245,236,6.00


In [164]:
#change names of the columns
df.columns = ['states','population','male','female','diff_bet_male_and_female','sex ratio','rural','urban','area','density','percent']
print(f"Number of observations in df {df.shape}")
df.head()

Number of observations in df (36, 11)


,states,population,male,female,diff_bet_male_and_female,sex ratio,rural,urban,area,density,percent
0,Uttar Pradesh,199812341,104480510,95331831,9148679,930,155111022,44470455,240928,828,16.50
1,Maharashtra,112374333,58243056,54131277,4111779,929,61545441,50827531,307713,365,9.28
2,Bihar,104099452,54278157,49821295,4456862,918,92075028,11729609,94163,1102,8.60
3,West Bengal,91276115,46809027,44467088,2341939,950,62213676,29134060,88752,1030,7.54
4,Madhya Pradesh,72626809,37612306,35014503,2597803,931,52537899,20059666,308245,236,6.00
